## 1. Get consumer key

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
consumer_key=os.getenv('MAERSK_CONSUMER_KEY')
''' set data folder path '''
proj_dir = os.path.abspath(os.pardir)
_dpath = os.path.join(
    proj_dir.split('wrangler/')[0],
    "wrangler/data/shipping/schedules"
)
print(f'data folder:\n{_dpath}')

data folder:
/home/nuwan/workspace/advantis/wrangler/data/shipping/schedules


## 2. Define input params

In [7]:
import pandas as pd

_inp_fpath = os.path.join(_dpath, "get_schedule_inputs.csv")
input_df = pd.read_csv(_inp_fpath)

params = {
    "collectionOriginCountryCode": input_df.iloc[0]['Originating place'],
    "collectionOriginCityName": input_df.iloc[0]['Originating port'],
    "deliveryDestinationCountryCode": input_df.iloc[0]['Delivery place'],
    "deliveryDestinationCityName": input_df.iloc[0]['Discharge port'],
    "vesselOperatorCarrierCode": input_df.iloc[0]['Vessel'],
}
params

{'collectionOriginCountryCode': 'LK',
 'collectionOriginCityName': 'Colombo',
 'deliveryDestinationCountryCode': 'TH',
 'deliveryDestinationCityName': 'Bangkok',
 'vesselOperatorCarrierCode': 'MAEU'}

## 3. Request schedules

In [8]:
import requests
import json

url = "https://api.maersk.com/products/ocean-products"
headers_a = {
    "Accept": "application/json",
    "Consumer-Key": consumer_key,  # Add your actual API key
    # "Content-Type": "application/json"
}

try:
    response = requests.get(
        url,
        params=params,
        headers=headers_a
    )
    
    # Check if request was successful
    if response.status_code == 200:
        print("Success! Response:")
        num_sched =len(response.json()['oceanProducts'][0]['transportSchedules'])
        print(f"recovered {num_sched} transport schedules")
    else:
        print(f"Error {response.status_code}: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

Success! Response:
recovered 4 transport schedules


## 4. Dump response to json file

In [9]:
_dump_fpath = os.path.join(_dpath,"maersk_response.json")
with open(_dump_fpath, 'w') as f:
    json.dump(response.json(), f)


## 5. Parse results

In [10]:
cols = ['schedule', 'departDateTime', 'arriveDateTime', 'transitTime',
        'originCountryCode', 'originUNPortCode', 'originCityName',
        'destinCountryCode', 'destinUNPortCode', 'destinCityName',
        'bookingFile', 'from', 'subject',
       ]

# for key in response.json()['oceanProducts'].keys():
#     print(key)
sched_df = pd.DataFrame(columns=cols)
sched_lst = []
i=0
for schedule in response.json()['oceanProducts'][0]['transportSchedules']:
    sched_dict = {}
    sched_dict['schedule'] = i
    sched_dict['departDateTime'] = schedule['departureDateTime']
    sched_dict['arriveDateTime'] = schedule['arrivalDateTime']
    sched_dict['transitTime'] = int(schedule['transitTime'])/(360)
    sched_dict['originCountryCode']=schedule['facilities']['collectionOrigin']['countryCode']
    sched_dict['originUNPortCode'] =schedule['facilities']['collectionOrigin']['UNLocationCode']
    sched_dict['originCityName'] = schedule['facilities']['collectionOrigin']['cityName']
    sched_dict['destinCountryCode']=schedule['facilities']['deliveryDestination']['countryCode']
    sched_dict['destinUNPortCode'] =schedule['facilities']['deliveryDestination']['UNLocationCode']
    sched_dict['destinCityName'] = schedule['facilities']['deliveryDestination']['cityName']
    sched_dict['bookingFile'] = input_df.iloc[0]['booking file']
    sched_dict['from'] = input_df.iloc[0]['from']
    sched_dict['subject'] = input_df.iloc[0]['subject']
    # sched_df = sched_df.append(sched_dict,ignore_index=True)
    sched_df = pd.concat([sched_df, pd.DataFrame([sched_dict])], ignore_index=True)
    i+=1
sched_df

,schedule,departDateTime,arriveDateTime,transitTime,originCountryCode,originUNPortCode,originCityName,destinCountryCode,destinUNPortCode,destinCityName,bookingFile,from,subject
0,0,2025-05-26T00:01:00,2025-06-19T14:00:00,98.330556,LK,LKCMB,Colombo,TH,THLKR,Bangkok,temp_booking_dbc6fc39-a0f4-42ea-bc65-3fd13fdb3...,paul.deniese@rezgateway.com,for Ref: 124_23_05_2025
1,1,2025-06-01T00:01:00,2025-06-26T14:00:00,102.330556,LK,LKCMB,Colombo,TH,THLKR,Bangkok,temp_booking_dbc6fc39-a0f4-42ea-bc65-3fd13fdb3...,paul.deniese@rezgateway.com,for Ref: 124_23_05_2025
2,2,2025-06-09T00:01:00,2025-07-03T14:00:00,98.330556,LK,LKCMB,Colombo,TH,THLKR,Bangkok,temp_booking_dbc6fc39-a0f4-42ea-bc65-3fd13fdb3...,paul.deniese@rezgateway.com,for Ref: 124_23_05_2025
3,3,2025-06-16T00:01:00,2025-07-10T14:00:00,98.330556,LK,LKCMB,Colombo,TH,THLKR,Bangkok,temp_booking_dbc6fc39-a0f4-42ea-bc65-3fd13fdb3...,paul.deniese@rezgateway.com,for Ref: 124_23_05_2025


## 6. Write results to JSON

In [11]:
_out_fpath = os.path.join(_dpath, "results_maersk_schedules.json")
# sched_df.to_excel(_fpath, index=False)
sched_df.to_json(_out_fpath,orient='table')